In [ ]:
import pandas as pd
import numpy as np
import json

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# Explore file

In [5]:
# enumerate excel sheets
horas_curso=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='horas_curso')
horas_curso.columns=horas_curso.columns.str.strip()
horas_curso=horas_curso[['ASIGNATURA','codigo', 'horas', 'cantidad de cursos', '# estudiantes',
       'Horas preparacion de clase']]
horas_curso.columns=['ASIGNATURA', 'codigo','horas', 'cantidad_de_cursos', 'total_estudiantes','horas_no_instruccion']
horas_curso.dropna(subset=['codigo'], inplace=True)
horas_curso['codigo']=horas_curso['codigo'].apply(lambda x: x[:9])
#horas_curso['ASIGNATURA']=horas_curso['ASIGNATURA'].apply(lambda x: x[10:])
dict_horas={i['codigo']:i for i in horas_curso.to_dict('records')}

#Save to json file for later use
json.dump(dict_horas, open('data/horas_curso.json', 'w'), indent=4)


In [6]:
idoneidad=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='docente_curso')


# remove final space from column names
idoneidad.columns = idoneidad.columns.str.strip()
idoneidad.rename(columns={'ASIGNATURA ':'ASIGNATURA'}, inplace=True)
idoneidad["ASIGNATURA"].fillna(method="ffill",inplace=True)
idoneidad["codigo"]=idoneidad["codigo"].apply(lambda x: x[:9])
idoneidad["ASIGNATURA"]=idoneidad["ASIGNATURA"].apply(lambda x: x[10:])
# replaces all zeros with NaN
idoneidad.replace(0, np.nan, inplace=True)

cols=idoneidad.columns
cols=list(cols)[1:]


idoneidad=idoneidad.melt(id_vars=['codigo', 'ASIGNATURA'], value_vars=cols, var_name='docente', value_name='idoneidad').dropna(subset=['idoneidad'])
idoneidad['idoneidad']=idoneidad.apply(lambda x: (x['codigo'], x['idoneidad']), axis=1)
idoneidad['docente']=idoneidad.docente.astype(int)
idoneidad=idoneidad.groupby('docente').agg(list)[['idoneidad']].reset_index()
idoneidad['idoneidad']=idoneidad['idoneidad'].apply(lambda x: {i[0]:i[1] for i in x})
idoneidad={i['docente']:i for i in idoneidad.to_dict('records')}
json.dump(idoneidad, open('data/idoneidad.json', 'w'), indent=4)

KeyError: 'ASIGNATURA'

In [7]:
labor_instruccional=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='labor_instruccional')
labor_instruccional.columns=labor_instruccional.columns.str.strip()
dict_labor={i['ID']:i for i in labor_instruccional.to_dict('records')}
json.dump(dict_labor, open('data/labor_instruccional.json', 'w'), indent=4)

# Horas reglamentarias

In [8]:
horas_reglamento=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='max_horas')
horas_reglamento.columns=horas_reglamento.columns.str.strip()
dict_horas_reglamento={i['n_cursos']:i for i in horas_reglamento.to_dict('records')}
json.dump(dict_horas_reglamento, open('data/horas_reglamento.json', 'w'), indent=4)

# cursos - requerimientos - tipo_sala

In [9]:
grupo_requerimiento=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='grupo_requerimiento')
grupo_requerimiento.columns=grupo_requerimiento.columns.str.strip()
grupo_requerimiento.dropna(subset=['codigo'], inplace=True)
grupo_requerimiento.rename(columns={'cursos':'nombre'}, inplace=True)
grupo_requerimiento={i['codigo']:i for i in grupo_requerimiento.to_dict('records')}
json.dump(grupo_requerimiento, open('data/grupo_requerimiento.json', 'w'), indent=4)

In [10]:
grupo_requerimiento=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='grupo_requerimiento')
grupo_requerimiento.columns=grupo_requerimiento.columns.str.strip()

grupo_requerimiento.dropna(subset=['codigo'], inplace=True)
dict_curso_codigo=grupo_requerimiento.set_index('cursos').to_dict()['codigo']


In [11]:
ubicacion_semestral=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='ubicacion_semestral')
ubicacion_semestral.columns=ubicacion_semestral.columns.str.strip()
ubicacion_semestral={i['codigo']:i for i in ubicacion_semestral.to_dict('records')}
json.dump(ubicacion_semestral, open('data/ubicacion_semestral.json', 'w'), indent=4)

# cursos - requerimientos - tipo_sala

In [18]:
capacidad_sala=pd.read_excel('Facultad_ingenieria3_OK.xlsx', sheet_name='capac_salon')
capacidad_sala.columns=capacidad_sala.columns.str.strip()
capacidad_sala={i['aula']:i for i in capacidad_sala.to_dict('records')}
json.dump(capacidad_sala, open('data/capacidad_sala.json', 'w'), indent=4)